In [1]:
# 泛式定義陣列
[x^2 for x in 3:10]

8-element Array{Int64,1}:
   9
  16
  25
  36
  49
  64
  81
 100

In [7]:
# 慢版的程式
# 運用 @time 巨集語法計算runtime

# 計算500個隨機值的平均, 重複1e6次, 並計算包含98%樣本的區間
using Statistics    # 載入套件
@time begin
    data = Float64[]
    for i in 1:1e6
        group = Float64[]
        for j in 1:5e2
            push!(group, rand())
        end
        push!(data, mean(group))
    end
    println("98% of the means lie in the estimated range: ", (quantile(data,0.01), quantile(data,0.99)))
end

98% of the means lie in the estimated range: (0.4699226996426578, 0.5298858888983439)
  5.745822 seconds (10.00 M allocations: 7.974 GiB, 6.35% gc time)


In [26]:
# 快版的程式
# 上面的程式浪費了太多記憶體空間, 導致程式較慢

using Statistics
@time begin
    data = [mean(rand(500)) for _ in 1:1e6]
    println("98% of the means lie in the estimated range: ", (quantile(data,0.01), quantile(data,0.99)))
end

98% of the means lie in the estimated range: (0.4699329311009187, 0.5300571919021461)
  1.044655 seconds (1.07 M allocations: 3.900 GiB, 14.74% gc time)


In [6]:
# Bubble sort
function bubbleSort!(a)       # 驚嘆號代表會改變a的內容
    n = length(a)
    for i in 1:n-1
        for j in 1:n-i
            if a[j] > a[j+1]
                a[j], a[j+1] = a[j+1], a[j]   # 賦值是同步進行的
            end
        end
    end
    return a
end

data = [65, 51, 32, 12, 23, 84, 68, 1]
bubbleSort!(data)
println("data = ", data)                      # In julia, arrays are passed by reference

data = [65, 51, 32, 12, 23, 84, 68, 1]
println("內建函數: data = ", sort(data))      # 除了sort(), 也有sort!()函數可供選擇

data = [1, 12, 23, 32, 51, 65, 68, 84]
內建函數: data = [1, 12, 23, 32, 51, 65, 68, 84]


In [7]:
# Roots of a polynomial

# f(x) = -10x^2 + 3x + 1
using Roots

function polynomialGenerator(a...)            # 輸入係數, 回傳簡單的多項式
    n = length(a) - 1
    poly = (x) -> sum([a[i+1]*x^i for i in 0:n])
    return poly
end

polynomial = polynomialGenerator(1, 3, -10)
zeroVals = find_zeros(polynomial, -10, 10)    # 在(-10, 10)之間找根
println("Zero of the function f(x): ", zeroVals)

┌ Info: Precompiling Roots [f2b01f46-fcfa-551c-844a-d8ac1e96c665]
└ @ Base loading.jl:1186


Zero of the function f(x): [-0.2, 0.5]


In [52]:
# Steady state of a Markov chain in several ways
using LinearAlgebra, StatsBase

# Transition probability matrix
P = [0.5 0.3 0.5;
     0.4 0.2 0.3;
     0.1 0.5 0.2]

# First way
piProb1 = (P^100)[:,1]
println("方法一: ", piProb1)

# Second way
A = vcat((P-I)[1:2,:], transpose(ones(3)))
b = transpose([0 0 1])
piProb2 = A\b
println("方法二: ", piProb2)

# Third way
eigVecs = eigvecs(P)
highestVec = eigVecs[:, findmax(abs.(eigvals(P)))[2]]
piProb3 = Array{Float64}(highestVec/norm(highestVec, 1))
println("方法三: ", piProb3)

# Fourth way
numInState = zeros(3)
state = 1
N = 10^6
for t in 1:N
    numInState[state] += 1
    global state = sample(1:3, weights(P[:,state]))
end
piProb4 = numInState/N
println("方法四: ", piProb4)

方法一: [0.4375, 0.3125, 0.25]
方法二: [0.4375; 0.3125; 0.25]
方法三: [0.4375, 0.3125, 0.25]
方法四: [0.438017, 0.312044, 0.249939]
